In [1]:
import pandas as pd
import json

## Gym

In [2]:
# Load the JSON file
file_path = 'gym.json'
with open(file_path, 'r') as file:
    data = json.load(file)

In [3]:
# Extract place names and coordinates
places = []
for result in data['results']:
    name = result['name']
    lat = result['geometry']['location']['lat']
    lng = result['geometry']['location']['lng']
    places.append({'name': name, 'latitude': lat, 'longitude': lng})

In [4]:
# Convert to DataFrame
df_gym = pd.DataFrame(places)

In [5]:
df_gym.head(1)

,name,latitude,longitude
0,Body Lab Fitness Barcelona,41.399642,2.119735


In [6]:
df_gym['category'] = 'gym'
df_gym.head(1)

,name,latitude,longitude,category
0,Body Lab Fitness Barcelona,41.399642,2.119735,gym


## Restaurant

In [7]:
# Load the JSON file
file_path = 'restaurant.json'
with open(file_path, 'r') as file:
    data = json.load(file)

In [8]:
# Extract place names and coordinates
places = []
for result in data['results']:
    name = result['name']
    lat = result['geometry']['location']['lat']
    lng = result['geometry']['location']['lng']
    places.append({'name': name, 'latitude': lat, 'longitude': lng})

In [9]:
# Convert to DataFrame
df_rest = pd.DataFrame(places)

In [10]:
df_rest.head(1)

,name,latitude,longitude
0,BOCCONI,41.399448,2.121785


In [11]:
df_rest['category'] = 'restaurant'
df_rest.head(1)

,name,latitude,longitude,category
0,BOCCONI,41.399448,2.121785,restaurant


## Museum

In [12]:
# Load the JSON file
file_path = 'museum.json'
with open(file_path, 'r') as file:
    data = json.load(file)

In [13]:
# Extract place names and coordinates
places = []
for result in data['results']:
    name = result['name']
    lat = result['geometry']['location']['lat']
    lng = result['geometry']['location']['lng']
    places.append({'name': name, 'latitude': lat, 'longitude': lng})

In [14]:
# Convert to DataFrame
df_museum = pd.DataFrame(places)

In [15]:
df_museum.head(1)

,name,latitude,longitude
0,Picasso Museum,41.385271,2.180947


In [16]:
df_museum['category'] = 'museum'
df_museum.head(1)

,name,latitude,longitude,category
0,Picasso Museum,41.385271,2.180947,museum


## Union All

In [17]:
df_places = pd.concat([df_gym, df_rest, df_museum], ignore_index=True)
df_places.head(1)

,name,latitude,longitude,category
0,Body Lab Fitness Barcelona,41.399642,2.119735,gym


## Meetup Data

In [18]:
file_path_meetup = 'meetup.json'
with open(file_path_meetup, 'r') as file:
    meetup_data = json.load(file)

In [19]:
# Extract place names and coordinates from the Google Maps links
meetup_places = []
for event in meetup_data:
    name = event['title']
    gmaps_link = event['gmaps_link']
    lat, lng = gmaps_link.split('query=')[1].split('%2C%20')
    meetup_places.append({'name': name, 'latitude': float(lat), 'longitude': float(lng)})

In [20]:
# Convert to DataFrame
df_meetup = pd.DataFrame(meetup_places)
df_meetup.head(1)

,name,latitude,longitude
0,Coffee Walk & Beach Sunset Hangout to Make New...,41.39655,2.194162


In [21]:
from geopy.distance import geodesic

In [22]:
nearest_places = []
for index, row in df_meetup.iterrows():
    defined_coordinate = (row['latitude'], row['longitude'])
    df_places['distance'] = df_places.apply(lambda r: geodesic(defined_coordinate, (r['latitude'], r['longitude'])).kilometers, axis=1)
    nearest = df_places.nsmallest(4, 'distance')[['name', 'latitude', 'longitude', 'distance']]
    nearest['from_place'] = row['name']
    nearest_places.append(nearest)

In [23]:
# Concatenate all the nearest places DataFrames
df_nearest_places = pd.concat(nearest_places, ignore_index=True)

In [25]:
df_nearest_places.head()

,name,latitude,longitude,distance,from_place
0,Design Museum of Barcelona,41.402453,2.188082,0.829684,Coffee Walk & Beach Sunset Hangout to Make New...
1,Elsa y Fred,41.389793,2.179749,1.419918,Coffee Walk & Beach Sunset Hangout to Make New...
2,Museu de la Xocolata,41.387356,2.181679,1.460345,Coffee Walk & Beach Sunset Hangout to Make New...
3,Casa Lolea Barcelona,41.388900,2.177156,1.656750,Coffee Walk & Beach Sunset Hangout to Make New...
4,Museum of the History of Catalonia,41.380804,2.185873,0.932970,"🎧 SILENT ECSTATIC BEACH - Music, Art & Meditat..."
